<a href="https://colab.research.google.com/github/sohaildua/PysparkLearning/blob/main/Manipulate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!pip install -U -q PyDrive --quiet 



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 49.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=426275a161b60211a78d6626566226a676f0f46fc15f15c1c976e308c588d665
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Manipulate").getOrCreate()
spark

In [6]:
from pyspark.sql.functions import concat_ws,col
data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]

columns = ["firstname","middlename","lastname","dob","gender","salary"]
df = spark.createDataFrame(data=data, schema = columns)
df2=df.select(concat_ws('_!_',df.firstname,df.middlename,df.lastname)
              .alias("FullName"),"dob","gender","salary")
df2.show(truncate=False)

+--------------------+----------+------+------+
|FullName            |dob       |gender|salary|
+--------------------+----------+------+------+
|James_!__!_Smith    |1991-04-01|M     |3000  |
|Michael_!_Rose_!_   |2000-05-19|M     |4000  |
|Robert_!__!_Williams|1978-09-05|M     |4000  |
|Maria_!_Anne_!_Jones|1967-12-01|F     |4000  |
|Jen_!_Mary_!_Brown  |1980-02-17|F     |-1    |
+--------------------+----------+------+------+



In [33]:
videos = spark.read.csv('youtubevideos.csv', inferSchema=True, header=True)

In [26]:
videos.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [34]:
videos.show(5, False)

+-----------+-------------+--------------------------------------------------------------+---------------------+-----------+------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [28]:
videos.limit(4).toPandas()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"""last week tonight trump presidency""|""last wee...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"""racist superman""|""rudy""|""mancuso""|""king""|""bac...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"""rhett and link""|""gmm""|""good mythical morning""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...


In [84]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [85]:

df_videos = videos.withColumn("views",videos["views"].cast(IntegerType())).\
withColumn("dislikes",videos["dislikes"].cast(IntegerType())).\
withColumn("likes",videos.likes.cast(IntegerType())).\
withColumn("trending_date",to_date(videos.trending_date,'yy.dd.mm'))
#withColumn("publish_time",to_timestamp(videos.publish_time,'yyyy-MM-dd HH:mm:ss')).\

In [86]:
df_videos.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: date (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [87]:
df_videos.show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|   2017-01-14|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           Fal

In [88]:
df_videos = df_videos.withColumn('publish_time_2',regexp_replace(df_videos.publish_time, 'T', ' '))

In [89]:
df_videos.show(4,False)

+-----------+-------------+--------------------------------------------------------------+---------------------+-----------+------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [90]:
df_videos = df_videos.withColumn('publish_time_2',regexp_replace(df_videos.publish_time_2, 'Z', ' '))

In [91]:
df_videos.show(4,False)

+-----------+-------------+--------------------------------------------------------------+---------------------+-----------+------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [96]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

df_videos = df_videos.withColumn("publish_time_3",to_timestamp(df_videos.publish_time_2,'yyyy-MM-dd HH:mm:ss.SSS'))

In [97]:
df_videos.show(1)

+-----------+-------------+--------------------+-------------+-----------+--------------------+---------------+------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+--------------------+-------------------+
|   video_id|trending_date|               title|channel_title|category_id|        publish_time|           tags| views|likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|      publish_time_2|     publish_time_3|
+-----------+-------------+--------------------+-------------+-----------+--------------------+---------------+------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+--------------------+-------------------+
|2kyS6SvSYSE|   2017-01-14|WE WANT TO TALK A...| CaseyNeistat|         22|2017-11-13T17:13:...|SHANtell martin|748374|57527|    2966|